In [39]:
import copy

import numpy as np
import pandas as pd
from collections import Counter, OrderedDict, defaultdict
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold, chi2, f_classif, RFECV
import matplotlib.pyplot as plt
from math import floor
from sklearn.preprocessing import LabelEncoder
import ipdb

import tqdm

# for Noe's model
from PMS2L.pegasos import SemiSupervisedSVM
from dataloaders.dataloader import *

# for Kseniia's model
from cotrain import SSLTrainSet
from cotrain import BinaryCoTrainer

import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

#увеличим дефолтный размер графиков
from pylab import rcParams

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go

In [48]:
all_datasets = ["STL10", "STL10_FFT2", "STL10_DWT2", "cifar10", "cifar10_FFT2" , "cifar10_DWT2"]

splits = [[6, 500 , 200], [10, 500 , 200], [20, 500 , 200], [50, 500 , 200], [100, 500 , 200]]

binary_list =[[1,2], [1,9]]

random_states = list(range(0, 101, 10))

In [41]:
# Data Frame
df = pd.DataFrame(columns=['Algorithm', 'Dataset', 'L_Size', 'U_Size', 'T_Size', 'Accuracy'])

In [54]:
rf_params = {'n_estimators': 50}


for dataset in all_datasets[5:6]:
    for binary in binary_list:
        for split in splits: 
            accuracy_list_ctr = np.zeros(len(random_states))   
            accuracy_list_ctr_bl = np.zeros(len(random_states))
            for idx, random_state in enumerate(random_states):
                print("\nDataset : " + dataset," Binary : " + str(binary), " Split : " +str(split))
               
                ######## KSENIIA algorithm ##########
                X1, Y1, X1u, X2, Y2, X2u, Xtest,Ytest  = get_samples(dataset=dataset,
                                                                     split=split,
                                                                     random_state=random_state,
                                                                     binary=binary,
                                                                     model='co-train')

                rf_params['random_state'] = random_state

                co_train = BinaryCoTrainer([RandomForestClassifier(**rf_params) for _ in range(2)],
                                           co_mode = "best")

                train_sets = [SSLTrainSet(X1, Y1, X1u), SSLTrainSet(X2, Y2, X2u)]
                co_train.fit(train_sets)

                co_scores = []
                for pred in co_train.predict(Xtest):
                    co_scores.append(accuracy_score(pred, Ytest))

                accuracy_list_ctr[idx] = np.mean(co_scores)

                #baseline
                model_bl1 = RandomForestClassifier(**rf_params)
                model_bl2 = RandomForestClassifier(**rf_params)        

                model_bl1.fit(X1, Y1)
                model_bl2.fit(X2, Y2)

                model_bl1_accuracy = accuracy_score(model_bl1.predict(Xtest), Ytest)
                model_bl2_accuracy = accuracy_score(model_bl2.predict(Xtest), Ytest)
                
                accuracy_list_ctr_bl[idx] = np.mean((model_bl1_accuracy,model_bl2_accuracy)) 
                #####################
                
            df = df.append({'Algorithm':'co-train', 'Dataset': dataset, 'L_Size' : split[0], 'U_Size' : split[1],'T_Size' : split[2], 'Accuracy' : np.mean(accuracy_list_ctr)}, ignore_index=True)
            df = df.append({'Algorithm':'co-train baseline', 'Dataset': dataset, 'L_Size' : split[0], 'U_Size' : split[1],'T_Size' : split[2], 'Accuracy' : np.mean(accuracy_list_ctr_bl)}, ignore_index=True)
#       


Dataset : cifar10_DWT2  Binary : [1, 2]  Split : [6, 500, 200]
Size of labeled data (3, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 2]  Split : [6, 500, 200]
Size of labeled data (3, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 2]  Split : [6, 500, 200]
Size of labeled data (3, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 2]  Split : [6, 500, 200]
Size of labeled data (3, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 2]  Split : [6, 500, 200]
Size of labeled data (3, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 2]  Split : [6, 500, 200]
Size of labeled data (3, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 2]


Dataset : cifar10_DWT2  Binary : [1, 2]  Split : [100, 500, 200]
Size of labeled data (50, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 2]  Split : [100, 500, 200]
Size of labeled data (50, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 2]  Split : [100, 500, 200]
Size of labeled data (50, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 2]  Split : [100, 500, 200]
Size of labeled data (50, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 9]  Split : [6, 500, 200]
Size of labeled data (3, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 9]  Split : [6, 500, 200]
Size of labeled data (3, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Bin


Dataset : cifar10_DWT2  Binary : [1, 9]  Split : [100, 500, 200]
Size of labeled data (50, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 9]  Split : [100, 500, 200]
Size of labeled data (50, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 9]  Split : [100, 500, 200]
Size of labeled data (50, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 9]  Split : [100, 500, 200]
Size of labeled data (50, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 9]  Split : [100, 500, 200]
Size of labeled data (50, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT2  Binary : [1, 9]  Split : [100, 500, 200]
Size of labeled data (50, 6348)
Size of unlabeled data (250, 6348)
Size of tets data (200, 6348)

Dataset : cifar10_DWT

In [55]:
df

,Algorithm,Dataset,L_Size,U_Size,T_Size,Accuracy
0,co-train,cifar10,6,500,200,0.335227
1,co-train baseline,cifar10,6,500,200,0.353864
2,co-train,cifar10,10,500,200,0.359318
3,co-train baseline,cifar10,10,500,200,0.348864
4,co-train,cifar10,20,500,200,0.361136
5,co-train baseline,cifar10,20,500,200,0.399773
6,co-train,cifar10,50,500,200,0.441818
7,co-train baseline,cifar10,50,500,200,0.457500
8,co-train,cifar10,100,500,200,0.510682
9,co-train baseline,cifar10,100,500,200,0.516136


In [56]:
df.to_csv('results/COTRAIN_TEST3_DS456.csv')